In [0]:
    %sql

CREATE OR REPLACE TABLE classes.gold.consumo_agua AS
SELECT DISTINCT
    UPPER(TRIM(`regiao`)) AS `regiao`,  
    UPPER(TRIM(`nome_cliente`)) AS `nome_cliente`,  
    UPPER(TRIM(`mes_referencia`)) AS `mes_referencia`,  
    CONCAT('R$', FORMAT_NUMBER(CAST(REPLACE(`valor_conta`, 'R$', '') AS DECIMAL(10,2)), 2)) AS `valor_conta`,  
    CONCAT('R$', FORMAT_NUMBER(CAST(REPLACE(`valor_imposto`, 'R$', '') AS DECIMAL(10,2)), 2)) AS `valor_imposto`,  
    CAST(`valor_conta_numeric` AS DOUBLE) AS `valor_conta_numeric`,  
    `consumo_por_m3`,  
    TO_DATE(`data_vencimento`, 'yyyy-MM-dd') AS `data_vencimento`,  
    TRIM(SPLIT(`localidade`, ' ')[0]) AS `bairro`,  
    CASE 
        WHEN SIZE(SPLIT(`localidade`, ' ')) = 1 THEN 'NÃO INFORMADO'
        ELSE TRIM(SUBSTRING(`localidade`, LENGTH(SPLIT(`localidade`, ' ')[0]) + 1))
    END AS `cidade`
FROM classes.silver.consumo_agua_clean
WHERE `consumo_por_m3` IS NOT NULL 
  AND `valor_conta` IS NOT NULL 
  AND `valor_imposto` IS NOT NULL;


In [0]:
%sql
-- Consultando todos os dados da tabela gold.consumo_agua
SELECT * FROM classes.gold.consumo_agua;


In [0]:
%sql
-- Criando a tabela gold.consumo_por_cidade para o mês com maior consumo
CREATE OR REPLACE TABLE classes.gold.consumo_por_cidade AS
SELECT 
    `regiao`, 
    `cidade`, 
    MAX(`consumo_por_m3`) AS `maior_consumo_por_m3`,
    `mes_referencia`
FROM classes.gold.consumo_agua
GROUP BY `regiao`, `cidade`, `mes_referencia`
ORDER BY `maior_consumo_por_m3` DESC
LIMIT 1;


In [0]:
%sql
-- Criando a tabela gold.maior_valor_imposto para o maior valor de imposto no Brasil
CREATE OR REPLACE TABLE classes.gold.maior_valor_imposto AS
SELECT 
    `regiao`, 
    `cidade`, 
    MAX(`valor_imposto`) AS `maior_imposto`,
    `mes_referencia`
FROM classes.gold.consumo_agua
GROUP BY `regiao`, `cidade`, `mes_referencia`
ORDER BY `maior_imposto` DESC
LIMIT 1;


In [0]:
%sql
-- Criando a tabela gold.mes_maior_consumo para o mês com maior consumo de água
CREATE OR REPLACE TABLE classes.gold.mes_maior_consumo AS
SELECT 
    `regiao`, 
    `cidade`, 
    MAX(`consumo_por_m3`) AS `maior_consumo`,
    `mes_referencia`
FROM classes.gold.consumo_agua
GROUP BY `regiao`, `cidade`, `mes_referencia`
ORDER BY `maior_consumo` DESC
LIMIT 1;


In [0]:
%sql
CREATE OR REPLACE TABLE classes.gold.fato_consumo_agua AS
SELECT
    /* Dimensões principais */
    regiao,
    cidade,
    nome_cliente,
    mes_referencia,
    bairro,

    /* Valores numéricos convertidos corretamente */
    CAST(REGEXP_REPLACE(valor_conta, 'R\\$|\\s', '') AS DECIMAL(10,2)) AS valor_conta_decimal,
    CAST(REGEXP_REPLACE(valor_imposto, 'R\\$|\\s', '') AS DECIMAL(10,2)) AS valor_imposto_decimal,
    consumo_por_m3,
    data_vencimento,

    /* Derivados úteis */
    (
        CAST(REGEXP_REPLACE(valor_conta, 'R\\$|\\s', '') AS DECIMAL(10,2)) +
        CAST(REGEXP_REPLACE(valor_imposto, 'R\\$|\\s', '') AS DECIMAL(10,2))
    ) AS valor_total,

    /* Datas */
    YEAR(data_vencimento) AS ano,
    MONTH(data_vencimento) AS mes_num,

    /* Indicador de alto consumo */
    CASE WHEN consumo_por_m3 > 100 THEN 1 ELSE 0 END AS alto_consumo

FROM classes.gold.consumo_agua;


In [0]:
%sql
SELECT * FROM classes.gold.fato_consumo_agua;


In [0]:
df = spark.table("classes.gold.fato_consumo_agua")
display(df)

from pyspark.sql import functions as F

df_top10 = (
    df.groupBy("regiao")
      .agg(F.max("valor_total").alias("maior_valor"))
      .orderBy(F.desc("maior_valor"))
      .limit(10)
      .withColumn(
          "maior_valor_formatado",
          F.concat(
              F.lit("R$ "),
              F.regexp_replace(
                  F.regexp_replace(
                      F.format_number("maior_valor", 2), ",", "X"
                  ),
                  "\\.", ","
              )
          )
      )
      .withColumn(
          "maior_valor_formatado",
          F.regexp_replace("maior_valor_formatado", "X", ".")
      )
)

display(df_top10)

df_top10.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("classes.gold.top10_maior_valor_FORMATADO")
